## 1. Customize API for Layer

In [ ]:
class CustomConv2D(tf.keras.layers.Layer):
    def __init__(self, filters, kernel_size, strides=(1, 1), padding='VALID', **kwargs):
        super(CustomConv2D, self).__init__(**kwargs)
        self.filters = filters
        self.kernel_size = kernel_size
        self.strides = (1, *strides, 1)
        self.padding = padding
    
    def build(self, input_shape):
        kernel_h, kernel_w = self.kernel_size
        input_dim = input_shape[-1]
        # Set weights in layer
        self.w = self.add_weight(name='kernel', shape=(kernel_h, kernel_w, input_dim, self.filters), initializer='glorot_uniform',
                                 trainable=True)
        # Set bias in layer
        self.b = self.add_weight(name='bias', shape=(self.filters, ), initializer='zeros', trainable=True)
    
    def call(self, inputs):
        x = tf.nn.conv2d(inputs, self.w, self.strides, padding=self.padding)
        x = tf.nn.bias_add(x, self.b)
        x = tf.nn.relu(x)
        return x

## 2. Customize Callback function

In [ ]:
class SavevModel(tf.keras.callbacks.Callback):
    def __init__(self, weights_file, monitor='loss', mode='min', save_weights_only=False):
        super(SaveModel, self).__init__()
        self.weights_file = weights_file
        self.monitor = monitor
        self.mode = mode
        self.save_weights_only = save_weights_only
        if mode == 'min':
            self.best = np.Inf
        else:
            self.best = -np.Inf
    
    def save_model(self):
        if self.save_weights_only:
            self.model.savee_weights(self.weights_file)
        else:
            self.model.save(self.weights_file)
    
    def on_epoch_end(self, epoch, logs=None):
        monitor_value = log.get(self.monitor)
        if self.mode == 'min' and monitor_value < self.best:
            self.save_model()
            self.best = monitor_value
        elif self.mode == 'max' and monitor_value > self.best:
            self.save_model()
            self.best = monitor_value

## 3. Customize Loss function

In [ ]:
def custom_categorical_crossentropy(y_true, y_pred):
    x = tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred)
    return x

## 4. Customize Categorical Accuracy function

In [ ]:
class CustomCategoricalAccuracy(tf.keras.metrics.Metric):
    def __init__(self, name='custom_categorical_accuracy', **kwargs):
        super(CustomCategoricalAccuracy, self).__init__(name=name, **kwargs)
        self.correct = self.add_weight('correct_numbers', initializer='zeros')
        self.total = self.add_weight(total_number, initializer='zeros')
    
    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.argmax(y_true, axis=-1)
        y_pred = tf.argmax(y_pred, axis=-1)
        values = tf.equal(y_true, y_pred)
        values = tf.cast(values, tf.float32)
        values_sum = tf.reduce_sum(values)
        num_values = tf.cast(tf.size(values), tf.float32)
        self.correct.assign_add(values_sum)
        self.total.assign_add(num_values)
    
    def result(self):
        return tf.math.divide_no_nan(self.correct, self.total)

    def reset_states(self):
        self.correct.assign(0.)
        self.total.assign(0.)